In [1]:
import json
import pandas as pd
# Укажите путь к вашему JSON-файлу
file_path = "../data/client_dataset.json"

    # Чтение JSON-файла
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    # Вывод содержимого файла
    data = pd.DataFrame(data)

In [2]:
from rapidfuzz import process, fuzz
import pandas as pd

def clean_and_reduce_skills(data, column_name='key_skills', threshold=80):
    """
    Очищает, нормализует и сокращает количество уникальных навыков в указанном столбце.
    
    Параметры:
        data (pd.DataFrame): Исходный DataFrame.
        column_name (str): Название столбца с навыками.
        threshold (int): Порог для объединения похожих навыков (0-100).
    
    Возвращает:
        list: Отсортированный и уменьшенный список уникальных навыков.
    """
    # Шаг 1: Удаляем пропущенные значения и разделяем навыки
    all_skills = data[column_name].dropna().str.split(',').sum()
    
    # Шаг 2: Приводим к нижнему регистру и удаляем лишние пробелы
    cleaned_skills = {skill.strip().lower() for skill in all_skills if skill.strip()}
    
    # Шаг 3: Объединяем похожие навыки
    reduced_skills = []
    while cleaned_skills:
        skill = cleaned_skills.pop()
        similar_skills = process.extract(skill, cleaned_skills, scorer=fuzz.ratio, score_cutoff=threshold)
        reduced_skills.append(skill)
        # Удаляем все найденные похожие навыки
        for match, _, _ in similar_skills:
            cleaned_skills.discard(match)
    
    # Шаг 4: Сортируем результат
    return sorted(reduced_skills)

In [4]:
import pandas as pd
import re
from collections import defaultdict

def normalize_russian_words(words):
    """
    Приводит русский текст к нормальной форме.
    """
    import pymorphy3
    morph = pymorphy3.MorphAnalyzer()
    return [morph.parse(word)[0].normal_form for word in words]

def add_features_to_dataframe(data, features, skills_column='key_skills'):
    """
    Добавляет столбцы для признаков и отмечает их наличие в ключевых навыках.

    Параметры:
        data (pd.DataFrame): Исходный DataFrame.
        features (list): Список признаков.
        skills_column (str): Название столбца с ключевыми навыками.

    Возвращает:
        pd.DataFrame: DataFrame с добавленными столбцами признаков.
    """
    # Удаляем дубликаты из списка признаков
    features = list(set(features))
    
    # Отделяем русские слова от остальных
    russian_words = [feature for feature in features if re.search(r'[а-яА-Я]', feature)]
    other_words = [feature for feature in features if feature not in russian_words]
    
    # Нормализуем русские слова
    normalized_russian_words = normalize_russian_words(russian_words)
    
    # Объединяем нормализованные русские слова с другими
    final_features = normalized_russian_words + other_words

    # Добавляем столбцы с признаками
    for feature in final_features:
        data[feature] = 0  # Инициализация столбцов с нулями
    
    # Проверяем навыки в key_skills построчно
    for index, row in data.iterrows():
        key_skills = str(row[skills_column]).lower().split(', ')
        for feature in final_features:
            if feature in key_skills:
                data.loc[index, feature] = 1  # Помечаем 1, если навык найден

    return data

# Набор признаков
features = [
    'внимательность', 'аналитические', 'креативный', 'ответственность', 'стрессоустойчивость', 'команда', 'инициативность', 'организованность', 'обучаемость', 'коммуникабельность', 'эмпатия', 'системный', 'адаптивность', 'самообучение', 'упорство', 'дисциплинированность', 'пунктуальность', 'дипломатичность', 'проактивность', 'результат', 'лидер', 'тайм-менеджмент', 'презентации', 'документы', 'наставничество', 'гибкость', 'этика', 'переговоры', 'конфликты', 'многозадачность', 'самоанализ', 'управление', 'точность', 'решения', 'развитие', 'управление', 'корреспонденция', 'клиенты', 'интеллект', 'коучинг', 'изменения', 'риски', 'четкость', 'логичность', 'планирование', 'c', 'c++', 'java', 'javascript', 'typescript', 'php', 'swift', 'kotlin', 'objective-c', 'ruby', 'scala', 'go', 'sql', 'pl/sql', 'bash', 'shell scripting', 'perl', 'r', 'haskell', 'groovy', 'visual basic', 'vbscript', 'lua', 'assembly', 'matlab', 'pascal', 'ada', 'fortran', 'rust', 'd', 'f#', 'dart', 'vhdl', 'verilog', 'cobol', 'clips', 'tcl', 'awk', 'prolog', 'lisp', 'smalltalk', 'clojure', 'erlang', 'elixir', 'assembler', 'react', 'angular', 'vue', 'spring', 'spring', 'django', 'flask', 'laravel', 'ruby', 'asp', 'node', 'express', 'koa', 'hapi', 'nest', 'quarkus', 'micronaut', 'play', 'jhipster', 'spark', 'akka', 'vert', 'dropwizard', 'gwt', 'google', 'sencha', 'vaadin', 'wicket', 'wicketstuff', 'primefaces', 'richfaces', 'nuxt', 'next', 'react', 'ionic', 'capacitor', 'phonegap', 'cordova', 'flutter', 'script', 'svelte', 'ember', 'backbone', 'knockout', 'mithril', 'aurelia', 'meteor', 'telerik', 'bootstrap', 'foundation', 'material-ui', 'tailwindcss', 'bulma', 'semantic', 'purecss', 'uikit', 'buefy', 'symfony', 'zend', 'yii', 'yii2', 'codeigniter', 'cakephp', 'fuelphp', 'phalcon', 'slim', 'lumen', 'phoenix', 'adonisjs', 'fastapi', 'pyramid', 'zope', 'web2py', 'bottle', 'tornado', 'cherry', 'fastify', 'grails', 'gradle', 'maven', 'ant', 'gulp', 'grunt', 'webpack', 'parcel', 'vite', 'rollup', 'lit', 'jest', 'mocha', 'chai', 'jasmine', 'enzyme', 'selenium', 'webdriverio', 'puppeteer', 'cypress', 'robot', 'playwright', 'testng', 'junit', 'nunit', 'xunit', 'pytest', 'tox', 'behave', 'allure', 'cucumber', 'robotium', 'espresso', 'appium', 'webdriver', 'taiko', 'redux', 'mobx', 'react-redux', 'vuex', 'pinia', 'ngrx', 'graphql', 'apollo', 'relay', 'featherjs', 'loopback', 'sails', 'strapi', 'keystonejs', 'sanic', 'aiohttp', 'flask-restplus', 'rest', 'falcon', 'restify', 'actix-web', 'beego', 'revel', 'echo', 'gin', 'buffalo', 'vega', 'blueprints', 'jinja2', 'mustache', 'handlebars', 'dot', 'swig', 'pug', 'twig', 'smarty', 'ejs', 'nunjucks', 'mjml', 'haml', 'eruby', 'liquid', 'razor', 'axios', 'fetch', 'lodash', 'underscore', 'moment', 'day', 'd3', 'three', 'chart', 'highcharts', 'echarts', 'ant-design', 'styled-components', 'emotion', 'jss', 'postcss', 'sass', 'less', 'scss', 'storybook', 'swr', 'vue-router', 'formik', 'yup', 'rxjs', 'rxswift', 'vis', 'webgl', 'pixi', 'amcharts', 'charts', 'fusioncharts', 'apexcharts', 'jspdf', 'pdfkit', 'pdf', 'numpy', 'scipy', 'matplotlib', 'seaborn', 'plotly', 'bokeh', 'altair', 'folium', 'geopandas', 'shapely', 'rasterio', 'fiona', 'openlayers', 'leaflet', 'turf', 'mapbox-gl', 'deck', 'chartkick', 'morris', 'animate', 'anime', 'gsap', 'phaser', 'p5', 'popmotion', 'qlikview', 'tableau', 'power', 'looker', 'kibana', 'elasticsearch', 'eslint', 'tslint', 'prettier', 'husky', 'lint-staged', 'babel', 'ts-node', 'nodemon', 'browserify', 'uglify', 'terser', 'sw-precache', 'workbox', 'pwa', 'lighthouse', 'aiohttp', 'urllib', 'socket', 'stomp', 'sockjs-client', 'mqtt', 'kafka-node', 'grpc', 'protobufjs', 'sharp', 'gm', 'imageio', 'pillow', 'pytorch', 'tensorflow', 'keras', 'scikit-learn', 'opencv', 'torchvision', 'torchaudio', 'nltk', 'spacy', 'gensim', 'transformers', 'huggingface', 'openai', 'ray', 'flask-socketio', 'celery', 'kombu', 'flower', 'redis', 'rq', 'dramatiq', 'huey', 'psycopg2', 'sqlalchemy', 'mysqlclient', 'pymysql', 'sqlite3', 'mongoengine', 'pymongo', 'motor', 'boto3', 'botocore', 'azure-sdk', 'google', 'firebase', 'pandasql', 'pyspark', 'hdfs', 'hive', 'impala', 'snowflake', 'databricks', 'dbt', 'bigquery', 'superset', 'dask', 'modin', 'pyarrow', 'fastparquet', 'csvkit', 'pyexcel', 'sqlmodel', 'orm', 'pony', 'peewee', 'dataset', 'petl', 'pandas', 'dash', 'scapy', 'mitmproxy', 'locust', 'jmeter', 'soapui', 'postman', 'insomniac', 'restsharp', 'retrofit', 'feign', 'okhttp', 'requests-html', 'scrapy', 'beautifulsoup4', 'lxml', 'parsel', 'pyquery', 'ruamel', 'pyyaml', 'jsonschema', 'cerberus', 'marshmallow', 'voluptuous', 'swagger', 'openapi', 'fast-json', 'jsonlite', 'rapidjson', 'simplejson', 'ujson', 'xlwings', 'xlutils', 'xlrd', 'pyxlsb', 'xlsxwriter', 'geopy', 'matplotlib-basemap', 'mplleaflet', 'h3', 'pyproj', 'rasterstats', 'openrouteservice', 'herepy', 'osmapi', 'osmnx', 'networkx', 'igraph', 'pyvis', 'vispy', 'pygal', 'vincent', 'hexbin', 'scattertext', 'spacy-streamlit', 'flair', 'wordcloud', 'textblob', 'vaderSentiment', 'polyglot', 'textacy', 'fasttext', 'bert', 'pyxlsb', 'win32com.client', 'reportlab', 'aws', 'gcp', 'azure', 'heroku', 'digitalocean', 'linode', 'vultr', 'oracle', 'ibm', 'openstack', 'rackspace', 'cloudflare', 'netlify', 'vercel', 'github', 'gitlab', 'lambda', 'azure', 'openwhisk', 'kubernetes', 'openshift', 'swarm', 'ecs', 'eks', 'gke', 'aks', 'nomad', 'mesos', 'marathon', 'cloud', 'paas', 'iaas', 'websphere', 'apache',
'jetty',
'jboss', 'glassfish', 'wildfly', 'beanstalk', 'vmware', 'proxmox', 'kvm', 'qemu', 'hyper-v', 'xen', 'vagrant', 'virtualbox', 'parallels', 'docker', 'podman', 'containerd', 'cri-o', 'rancher', 'k3s', 'harbor', 'helm', 'terraform', 'ansible', 'chef', 'puppet', 'saltstack', 'packer', 'concourse', 'teamcity', 'jenkins', 'bamboo', 'travisci', 'circleci', 'gitlab-ci', 'github-actions', 'azure-devops', 'bitbucket', 'appveyor', 'buddy', 'spinnaker', 'codefresh', 'semaphoreci', 'fluxcd', 'argo', 'cloudbees', 'app', 'service', 'zapier', 'mendix', 'outsystems', 'power', 'autodesk', 'gym', 'selenium', 'sauce', 'browserstack', 'crossbrowsertesting', 'lambdatest', 'testim', 'testproject', 'perfecto', 'applitools', 'sentry', 'datadog', 'relic', 'dynatrace', 'appdynamics', 'jaeger', 'opentelemetry', 'cloudwatch', 'stackdriver', 'graylog', 'papertrail', 'loggly', 'fluentd', 'vector', 'filebeat', 'winston', 'morgan', 'bunyan', 'русский', 'английский', 'немецкий', 'французский', 'испанский', 'итальянский', 'китайский', 'японский', 'корейский', 'азербайджанский', 'голландский', 'tensorflow_serving', 'pytorch_lightning', 'pytorch_hub', 'huggingface_hub', 'ray', 'scikit-learn', 'transformers', 'keras', 'nltk', 'spacy', 'gensim', 'fasttext', 'wordcloud', 'textblob', 'vaderSentiment', 'bert', 'flair', 'polyglot', 'tensorflow', 'opencv', 'torchvision', 'torchaudio', 'pytorch', 'skitlearn', 'catboost', 'lightgbm', 'xgboost', 'shap', 'lime', 'ml', 'weka', 'mlflow', 'h2o.ai', 'datarobot'
]

# Добавляем столбцы с признаками
processed_data = add_features_to_dataframe(data, features)

C:\Users\Артём\AppData\Local\Temp\ipykernel_6596\2676990201.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0  # Инициализация столбцов с нулями
C:\Users\Артём\AppData\Local\Temp\ipykernel_6596\2676990201.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0  # Инициализация столбцов с нулями
C:\Users\Артём\AppData\Local\Temp\ipykernel_6596\2676990201.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

C:\Users\Артём\AppData\Local\Temp\ipykernel_6596\2676990201.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0  # Инициализация столбцов с нулями
C:\Users\Артём\AppData\Local\Temp\ipykernel_6596\2676990201.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[feature] = 0  # Инициализация столбцов с нулями
C:\Users\Артём\AppData\Local\Temp\ipykernel_6596\2676990201.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

In [5]:
processed_data['c++'].sum()

352